Librerias a trabajar con Google Drive & Google Sheets

In [1]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
import gspread
import pprint
import pandas as pd
import numpy as np

In [2]:
#Documentación
ruta_accesos = "D:/SITEC/permisos/sitecsolucionesyservicios-67cee8fd8c65.json"
alcances = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']


#autenticación
credenciales = service_account.Credentials.from_service_account_file(ruta_accesos,scopes = alcances) 

#construir el servicio de google drive
drive = build('drive', 'v3', credentials= credenciales)

# Ruta al archivo JSON de credenciales de la cuenta de servicio
ruta_accesos = "D:/SITEC/permisos/sitecsolucionesyservicios-67cee8fd8c65.json"
alcances = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

# Autenticación
credenciales = service_account.Credentials.from_service_account_file(ruta_accesos, scopes=alcances)

# Construir el servicio de Google Drive
drive = build('drive', 'v3', credentials=credenciales)

In [ ]:
#Funcion de lectura de documentos de google drive:
def lista_carpetas(service):
    query = "mimeType='application/vnd.google-apps.folder'" #tipo de archivo de google Drive (Multipurpose Internet Mail Extensions-MIME)
    resultados = service.files().list(q=query).execute()
    carpetas = pd.DataFrame(resultados.get('files', []))
    
    return carpetas[['name','id']]

carpetas = lista_carpetas(drive)
dimh = len(carpetas)
print(carpetas.iloc[:,:])


In [4]:
#funcion para extraer los nombres de los archivos dentro de las carpetas
def archivos_en_carpeta(service, folder_id):
    query = f"'{folder_id}' in parents"
    resultados = service.files().list(q=query).execute()
    archivos = pd.DataFrame(resultados.get('files', []))
    return archivos['name'].tolist() if 'name' in archivos else []

propio+gpt

In [5]:
#Generamos encabezados
h = carpetas['name'].values

#creamos un diccionario para almacenar los archivos por carpeta
archivos_dict = {nom_carpeta: archivos_en_carpeta(drive,folder_id) for nom_carpeta,folder_id in zip(carpetas['name'],carpetas['id'])}
#convertimos el diccionario en un dataframe

max_files = max(len(files) for files in archivos_dict.values())
for key in archivos_dict:
    archivos_dict[key].extend([None]*(max_files-len(archivos_dict[key])))   #se crea el dataframe con la cantidad de archivos máximos de una carpeta, consecutivo los demas con valor None.

datos = pd.DataFrame(archivos_dict)
    

In [ ]:
print(datos)

Transformacion de archivos .xlsx a archivos .googlesheet


In [ ]:
#convertiremos una unidad (un archivo) de xlsx. a google sheet para posteriormente aplicarlo de forma general
 
#Del anterior código requerimos: file_id & nombre_hoja_calculo 
 
def conversor_xlsx_to_gs(servicio, file_id, nom_hoja_calculo):
    #configuramos los metadatos del nuevo archivo para que cambie el formato del archivo
    metadata_archivo = {
        'name' : nom_hoja_calculo,
        'mimeType' : 'application/vnd.google-apps.spreadsheets'
    }
#Realizamos la copia con las conversiones
    archivo_google_sheet = servicio.files().copy(
        fileId = file_id,
        body = metadata_archivo
    ).execute()
 
    return archivo_google_sheet


id_archivo = '1OVmEqrA63SzrM8XXcjLhUz3XfaPnAiU4'    
nombre_archivo = 'Abril2024edades.xlsx'
#guardamos el archivo en un objeto
archivo_convertido = conversor_xlsx_to_gs(drive,id_archivo,nombre_archivo)
print(archivo_convertido)


In [25]:
# Función para verificar permisos del archivo
def verificar_permisos(servicio, file_id, email_usuario):
    permisos = servicio.permissions().list(fileId=file_id).execute()
    for permiso in permisos.get('permissions', []):
        if permiso.get('emailAddress') == email_usuario:
            return True
    return False

# Función para convertir un archivo .xlsx a Google Sheets
def conversor_xlsx_to_gs(servicio, file_id, nom_hoja_calculo):
    # Configuramos los metadatos del nuevo archivo para que cambie el formato del archivo
    metadata_archivo = {
        'name': nom_hoja_calculo,
        'mimeType': 'application/vnd.google-apps.spreadsheet'
    }

    try:
        # Realizamos la copia con las conversiones
        archivo_google_sheet = servicio.files().copy(
            fileId=file_id,
            body=metadata_archivo
        ).execute()
    except Exception as e:
        print(f"Error al intentar copiar el archivo: {e}")
        return None
    
    return archivo_google_sheet

# Función para obtener el ID de un archivo dado su nombre
def obtener_id_archivo(service, nombre_archivo):
    results = service.files().list(
        q=f"name='{nombre_archivo}'",
        spaces='drive',
        fields='files(id, name, mimeType)').execute()
    items = results.get('files', [])
    
    if not items:
        print(f"No se encontró ningún archivo con el nombre {nombre_archivo}.")
        return None
    else:
        print(f"Archivo encontrado: {items[0]['name']} con MIME type: {items[0]['mimeType']}")
        return items[0]['id']

# Uso del servicio de Google Drive autenticado
# servicio_drive = autenticar_google_drive()  # Supongamos que ya tienes el servicio autenticado

# Nombre del archivo del que quieres obtener el ID
nombre_archivo = 'Abril2024edades.xlsx'

# Obtener el ID del archivo y guardarlo en un objeto
file_id = obtener_id_archivo(drive, nombre_archivo)

if file_id:
    print(f"El ID del archivo es: {file_id}")

    # Verificar permisos
    email_cuenta_servicio = 'sitecsolucionesyservicios@sitecsolucionesyservicios.iam.gserviceaccount.com'
    if verificar_permisos(drive, file_id, email_cuenta_servicio):
        # Convertir el archivo .xlsx a Google Sheets
        archivo_convertido = conversor_xlsx_to_gs(drive, file_id, nombre_archivo)
        if archivo_convertido:
            print("Archivo convertido exitosamente:")
            print(archivo_convertido)
        else:
            print("No se pudo convertir el archivo.")
    else:
        print(f"La cuenta de servicio {email_cuenta_servicio} no tiene los permisos necesarios para modificar el archivo.")
else:
    print("No se pudo obtener el ID del archivo.")





Archivo encontrado: Abril2024edades.xlsx con MIME type: application/vnd.openxmlformats-officedocument.spreadsheetml.sheet
El ID del archivo es: 1OVmEqrA63SzrM8XXcjLhUz3XfaPnAiU4
La cuenta de servicio sitecsolucionesyservicios@sitecsolucionesyservicios.iam.gserviceaccount.com no tiene los permisos necesarios para modificar el archivo.


Eliminar el archivo .xslx 

In [22]:
def eliminar_archivo(servicio,file_id):
    servicio.files().delete(fileId = file_id).execute()
    print(f"archivo con ID: {file_id} eliminado")

Obtener ubicacion archivo original

In [23]:
def carpeta_padre(servicio,file_id):
    archivo = servicio.files().get(fileID = file_id, fields='parents').execute()
    return archivo.get('parents')[0] if archivo.get('parents') else None

Mover archivo a la ubicacion original

In [ ]:
def mover_archivo(servicio,file_id,carpeta_id):
    archivo = servicio.files.get(fileID = file_id, fields = 'parents').execute
    ubicacion_actual = archivo.get('parents')
    
    if ubicacion_actual:
        for carpeta in ubicacion_actual:
            servicio.files().get(FileId = file_id, removeParents = carpeta).execute()
    #Agregar archivo a la ubicacion actual
    servicio.files().update(fileId = file_id, addParents=carpeta_id).execute()
    print(f"el archivo con ID:{file_id} ha sido movido a la carpeta con ID: {carpeta_id}")    
    
    


In [12]:
#El archivo debe pertenecer a una carpeta, de no ser asi no detectará el archivo
def obtener_id_archivo(service, nombre_archivo):
    results = service.files().list(
        q=f"name='{nombre_archivo}'",
        spaces='drive',
        fields='files(id, name)').execute()
    items = results.get('files', [])
    
    if not items:
        print(f"No se encontró ningún archivo con el nombre {nombre_archivo}.")
        return None
    else:
        return items[0]['id']

# Uso del servicio de Google Drive autenticado
# servicio_drive = autenticar_google_drive()  # Supongamos que ya tienes el servicio autenticado

# Nombre del archivo del que quieres obtener el ID
nombre_archivo = 'Abril2024edades.xlsx'

# Obtener el ID del archivo y guardarlo en un objeto
file_id = obtener_id_archivo(drive, nombre_archivo)

if file_id:
    print(f"El ID del archivo es: {file_id}")


El ID del archivo es: 1OVmEqrA63SzrM8XXcjLhUz3XfaPnAiU4


In [38]:
#definimos los datos de forma local:
   
id_archivo = '1OVmEqrA63SzrM8XXcjLhUz3XfaPnAiU4'    
nombre_archivo = 'Abril2024edades.xslx'
#guardamos el archivo en un objeto
archivo_convertido = conversor_xlsx_to_gs(drive,id_archivo,nombre_archivo)
print(archivo_convertido)

#Obtenemos la carpeta padre (original)
carpeta_padre_id = carpeta_padre(drive,id_archivo)
print(carpeta_padre_id)

#Eliminamos el archivo viejo
eliminar_archivo(drive,id_archivo)

#mover archivo convertido a la carpeta original
mover_archivo(drive,id_archivo, carpeta_padre_id)



if carpeta_padre_id:
    mover_archivo(drive,archivo_convertido['id'], carpeta_padre_id)
else:
    print("el archivo original no tenia una carpeta padre")    

TypeError: Got an unexpected keyword argument fileID